# Function / Tool Calling

In [2]:
import os
import requests
import json

In [ ]:
# A function to make a call to openweathermap and get current weather

def get_weather(city_name):
    owmkey=os.environ['OWM']
    api = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={owmkey}"

    response = requests.get(api)
    response = response.content.decode()
    response = json.loads(response)
    output = {"weather":response['weather'][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}
    return output

In [10]:
get_weather("mumbai")

{'weather': 'clear sky', 'temperature': 301.14, 'unit': 'kelvin'}

In [44]:
import re
from openai import OpenAI
client = OpenAI()

def clean_response(text):
   text = re.sub("```python","",text) 
   text = re.sub("```","",text)
   return text.strip()

def code_commentor(input_file):
    # step 1: load the content file

    with open(input_file) as file:
        code = file.read()

    # step 2: get code with comments and doc

    prompt = f""" you are an expert programmer, code reviewer and commentor.
    you need to read the below code in triple backticks and add comments and docstring
    Do not modify the code or logic, only add comments and doc string.
    Only provide code as output, do not add any additional text or explanation
    Code:
    ```{code}``
    """
    messages = [{'role':'user','content':prompt},]
    response = client.chat.completions.create(model='gpt-4o', messages=messages, temperature=0)
    response = response.choices[0].message.content
    response = clean_response(response)

    # step 3: export the updated code file.
    output_file = input_file[:-3]+"_c"+".py"
    with open(output_file,'w') as file:
        file.write(response)
    print("file successfully exported")
    return f"file is successfully exported after adding comment to {output_file}"


In [45]:
tools = [{"type":"function",
          "function":{
              "name": "get_current_weather",
              "description":"This function is used to get current weather information for a given city.",
              "parameters":{
                  "type":"object",
                  "properties":{
                      "city_name":{"type":"string","description":"name of location/city e.g. mumbai, new york"},
                  }
              },
              "required":['city_name'], }},
              {"type":"function",
          "function":{
              "name": "write_code_comment_docs",
              "description":"This function is used to add comments and docstring for a given .py file",
              "parameters":{
                  "type":"object",
                  "properties":{
                      "input_file":{"type":"string","description":" name of a python file (only .py), e.g. abc.py "},
                  }
              },
              "required":['input_file'], }},
              ]

In [46]:
from openai import OpenAI
client = OpenAI()
def generate_response(prompt):
    messages = [{'role':'system','content':"you are a helpful assistant, always provide information when possible and use tool when applicable."},
                {'role':'user','content':prompt},]
    first_response = client.chat.completions.create(model='gpt-4o',messages=messages,temperature=0,
                                                    tools=tools,tool_choice='auto')
    print(first_response.choices[0])
    tool_call = first_response.choices[0].message.tool_calls

    if tool_call:
        tool_map = {'get_current_weather':get_weather,
                    'write_code_comment_docs':code_commentor}
        messages.append(first_response.choices[0].message)

        for tool in tool_call:
            tool_name = tool.function.name
            tool_fun = tool_map[tool_name]
            tool_args = json.loads(tool.function.arguments)
            tool_output = tool_fun(**tool_args)
            messages.append({"tool_call_id":tool.id,
                             "role":"tool",
                             "name":tool_name,
                             "content":json.dumps(tool_output)   })
        print(messages)

        second_response = client.chat.completions.create(model='gpt-4o',messages=messages,temperature=0)
        return second_response.choices[0].message.content
    else:
        return first_response.choices[0].message.content

In [36]:
generate_response("explain quantum computing in simpler terms.")

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Quantum computing is a type of computing that uses the principles of quantum mechanics to process information. Here's a simplified explanation:\n\n1. **Bits vs. Qubits**: Traditional computers use bits as the smallest unit of data, which can be either 0 or 1. Quantum computers use qubits, which can be both 0 and 1 at the same time, thanks to a property called superposition.\n\n2. **Superposition**: Imagine a spinning coin. While it's spinning, it's not just heads or tails; it's a mix of both. Similarly, a qubit can be in a state of 0, 1, or both simultaneously.\n\n3. **Entanglement**: This is a special connection between qubits. When qubits are entangled, the state of one qubit can depend on the state of another, no matter how far apart they are. It's like having two dice that always show the same number when rolled, even if they're miles apart.\n\n4. **Quantum Gates**: These are like the logic 

"Quantum computing is a type of computing that uses the principles of quantum mechanics to process information. Here's a simplified explanation:\n\n1. **Bits vs. Qubits**: Traditional computers use bits as the smallest unit of data, which can be either 0 or 1. Quantum computers use qubits, which can be both 0 and 1 at the same time, thanks to a property called superposition.\n\n2. **Superposition**: Imagine a spinning coin. While it's spinning, it's not just heads or tails; it's a mix of both. Similarly, a qubit can be in a state of 0, 1, or both simultaneously.\n\n3. **Entanglement**: This is a special connection between qubits. When qubits are entangled, the state of one qubit can depend on the state of another, no matter how far apart they are. It's like having two dice that always show the same number when rolled, even if they're miles apart.\n\n4. **Quantum Gates**: These are like the logic gates in classical computers but for qubits. They manipulate qubits through operations that

In [37]:
generate_response("How is the weather in london today and also for mumbai.")

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hDn1pgDJe7BHgAHDGEFxDUYP', function=Function(arguments='{"city_name": "london"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_6l3S10PqTNF4I4Z3pYoQr5Qg', function=Function(arguments='{"city_name": "mumbai"}', name='get_current_weather'), type='function')]))
[{'role': 'user', 'content': 'How is the weather in london today and also for mumbai.'}, ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hDn1pgDJe7BHgAHDGEFxDUYP', function=Function(arguments='{"city_name": "london"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_6l3S10PqTNF4I4Z3pYoQr5Qg', function=Functio

'The current weather in London is clear with a temperature of approximately 283.71 Kelvin (about 10.56°C or 51°F). In Mumbai, the weather is also clear, with a temperature of approximately 301.49 Kelvin (about 28.34°C or 83°F).'

In [41]:
print(generate_response("Tell me about the city Jakarta, what is the weather in Jakarta today."))

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_emud3JWCwvcNWjMBZkkZdqtd', function=Function(arguments='{"city_name":"Jakarta"}', name='get_current_weather'), type='function')]))
[{'role': 'system', 'content': 'you are a helpful assistant, always provide information when possible and use tool when applicable.'}, {'role': 'user', 'content': 'Tell me about the city Jakarta, what is the weather in Jakarta today.'}, ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_emud3JWCwvcNWjMBZkkZdqtd', function=Function(arguments='{"city_name":"Jakarta"}', name='get_current_weather'), type='function')]), {'tool_call_id': 'call_emud3JWCwvcNWjMBZkkZdqtd', 'role': 'tool', 'name': 'get_current_weather', 'co

In [47]:
generate_response("can you add comments and docstrings to the python file msbom.py ")

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Jf5nqtomTae6P3Bh0tIaZffs', function=Function(arguments='{"input_file":"msbom.py"}', name='write_code_comment_docs'), type='function')]))
file successfully exported
[{'role': 'system', 'content': 'you are a helpful assistant, always provide information when possible and use tool when applicable.'}, {'role': 'user', 'content': 'can you add comments and docstrings to the python file msbom.py '}, ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Jf5nqtomTae6P3Bh0tIaZffs', function=Function(arguments='{"input_file":"msbom.py"}', name='write_code_comment_docs'), type='function')]), {'tool_call_id': 'call_Jf5nqtomTae6P3Bh0tIaZffs', 'role': 'tool', 

'I have added comments and docstrings to the `msbom.py` file. If you have any specific questions or need further assistance with the code, feel free to ask!'